In [2]:
from google.colab import files


uploaded = files.upload()


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle datasets download -d tarandeep97/xraysnormalcovid

 95% 145M/152M [00:03<00:00, 41.3MB/s]
100% 152M/152M [00:03<00:00, 40.7MB/s]


In [4]:
!mkdir chest-xray-pneumonia

!unzip xraysnormalcovid.zip -d chest-xray-pneumonia

Archive:  xraysnormalcovid.zip
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_155.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_156.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_157.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person30_bacteria_158.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person31_bacteria_160.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person33_bacteria_169.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/bacterial/person33_bacteria_172.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/covid/nejmc2001573_f1b.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/covid/nejmoa2001191_f4.jpeg  
  inflating: chest-xray-pneumonia/covid-normal-pneumonia/test/covid/nejmoa2001

In [0]:
TRAIN_PATH = './chest-xray-pneumonia/covid-normal-pneumonia/train'
VAL_PATH = './chest-xray-pneumonia/covid-normal-pneumonia/val'
TEST_PATH = './chest-xray-pneumonia/covid-normal-pneumonia/test'

In [6]:
import pathlib
data_dir = pathlib.Path(TRAIN_PATH)
image_count = len(list(data_dir.glob('*/*')))
print(image_count)

478


In [0]:
#Helper libraries
import pandas as pd
import numpy as np

#Frameworks and APIs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D,GlobalAveragePooling2D, Dense, Dropout, Softmax, Input, Flatten,BatchNormalization, Activation 
from keras.optimizers import adam, RMSprop, SGD

Using TensorFlow backend.


In [0]:
np.random.seed(1)
tf.random.set_seed(2)

In [12]:
image_gen = ImageDataGenerator(
                rescale = 1./255,
                featurewise_std_normalization=True,
                samplewise_std_normalization=True,
                vertical_flip = True,
                horizontal_flip = True)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
BATCH_SIZE= 14
IMG_H = 224
IMG_W = 224
STEP_PER_EPOCH = np.ceil(image_count / BATCH_SIZE)

In [14]:
train_batches = image_gen.flow_from_directory(
                                 TRAIN_PATH,
                                 target_size=(IMG_H, IMG_W),
                                 classes=['covid','normal', 'bacterial','viral'],
                                 batch_size=BATCH_SIZE)
val_batches = image_gen.flow_from_directory(
                                 VAL_PATH,
                                 target_size=(IMG_H, IMG_W),
                                 classes=['covid','normal', 'bacterial','viral'],
                                 batch_size=BATCH_SIZE)
test_batches = image_gen.flow_from_directory(
                                 TEST_PATH,
                                 target_size=(IMG_H, IMG_W),
                                 classes=['covid','normal', 'bacterial','viral'],
                                 batch_size=BATCH_SIZE)


Found 478 images belonging to 4 classes.
Found 56 images belonging to 4 classes.
Found 28 images belonging to 4 classes.


In [0]:
def model(learning_rate):
        model = Sequential()
        model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same", input_shape=(IMG_H,IMG_W,3)))
        model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same"))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(BatchNormalization())

        model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(rate=0.4))

        model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"))
        model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Dropout(rate=0.4))

        model.add(Dropout(rate=0.4))
        model.add(Flatten())

        model.add(Dense(1024, activation="relu"))
        model.add(BatchNormalization())
        model.add(Dropout(rate=0.2))
        model.add(Dense(4, activation="relu"))
    
        model.compile(optimizer = RMSprop(lr = learning_rate),
                  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
    
        return model

In [17]:
model = model(0.0001)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 64)       

In [18]:
total_epochs = 30

history_fine = model.fit(train_batches,
                         epochs = total_epochs,
                         shuffle=True, 
                         initial_epoch=0,
                         validation_data = val_batches)

Epoch 1/3


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


35/35 [==============================] - 18s 525ms/step - loss: 1.1683 - accuracy: 0.6109 - val_loss: 2.0062 - val_accuracy: 0.4107
Epoch 2/3
35/35 [==============================] - 12s 351ms/step - loss: 0.8829 - accuracy: 0.7594 - val_loss: 1.5496 - val_accuracy: 0.4643
Epoch 3/3
35/35 [==============================] - 12s 354ms/step - loss: 0.6799 - accuracy: 0.7741 - val_loss: 4.0309 - val_accuracy: 0.2500


In [19]:
model.evaluate(test_batches)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2/2 [==============================] - 1s 261ms/step


[4.429477691650391, 0.25]

In [0]:
print(model.metrics_names)

['loss', 'accuracy']


In [0]:
!pip install -U ipython
!pip install tensorflowjs

     |████████████████████████████████| 788kB 3.5MB/s 
     |████████████████████████████████| 358kB 15.4MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.14.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


     |████████████████████████████████| 61kB 2.2MB/s 
     |████████████████████████████████| 92kB 5.3MB/s 
     |████████████████████████████████| 144.4MB 107kB/s 
     |████████████████████████████████| 256kB 38.6MB/s 
     |████████████████████████████████| 921kB 22.4MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=18e5d53ad36966105f0e39ec1b41bbca5b50599bae3ca424394e5fe0af0d7283
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: ipython 7.14.0 has requirement prompt-toolkit!=3.0.0,!=3.0.1,<3.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.14.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: prompt-toolkit 3.0.5
    Unins

In [0]:
!mkdir model_85

In [0]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, "./model_85")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [0]:
!zip -r ./model_85.zip ./model_85

  adding: model_85/ (stored 0%)
  adding: model_85/group1-shard7of50.bin (deflated 7%)
  adding: model_85/group1-shard26of50.bin (deflated 7%)
  adding: model_85/group1-shard5of50.bin (deflated 7%)
  adding: model_85/group1-shard12of50.bin (deflated 7%)
  adding: model_85/group1-shard19of50.bin (deflated 7%)
  adding: model_85/group1-shard49of50.bin (deflated 7%)
  adding: model_85/group1-shard17of50.bin (deflated 7%)
  adding: model_85/group1-shard13of50.bin (deflated 7%)
  adding: model_85/group1-shard33of50.bin (deflated 7%)
  adding: model_85/group1-shard45of50.bin (deflated 7%)
  adding: model_85/group1-shard34of50.bin (deflated 7%)
  adding: model_85/group1-shard35of50.bin (deflated 7%)
  adding: model_85/group1-shard6of50.bin (deflated 7%)
  adding: model_85/group1-shard50of50.bin (deflated 7%)
  adding: model_85/group1-shard24of50.bin (deflated 7%)
  adding: model_85/group1-shard2of50.bin (deflated 7%)
  adding: model_85/group1-shard43of50.bin (deflated 7%)
  adding: model_85/g